# 18.06 pset 9 - Solutions

## Problem 1

(For this problem, you might want to review the analysis of Fibonacci numbers from section 6.2 of the text.  You can also google "Fibonacci matrix" to find lots of similar info online at various levels of sophistication.)

Consider the ["Tetranacci number" sequence](https://oeis.org/A000078) $t_n$:

$$
0, 0, 0, 1, 1, 2, 4, 8, 15, 29, 56, 108, 208, 401, 773, 1490, 2872, 5536, 10671, 20569, 39648, 76424, 147312, 283953, 547337
$$

The first four terms are $t_1,t_2,t_3,t_4 = 0, 0, 0, 1$, and after that each number is the sum of *four* preceding terms:

$$
t_n = t_{n-4} + t_{n-3} + t_{n-2} + t_{n-1}
$$

## (a)

Write this *recurrence relation* in matrix form $\vec{t}_n = A \vec{t}_{n-1}$ where $\vec{t}_n = (t_n, t_{n-1}, t_{n-2}, t_{n-3})$ for some matrix $A$.

Check that you have the right matrix by computing `A^16 * [1,0,0,0]` in Julia and comparing to the sequence terms given above.  (Make sure that you compare to the right terms.  Which terms should this give?)

## (b)

Find the eigenvalues of $A$ numerically via `eigvals(A)`.  Using these, predict the what the ratio $t_n / t_{n-1}$ tends to for large $n$.

Check your answer from numerically by computing `t = big(A)^n * [1,0,0,0]` in Julia to get the $\vec{t}_{n+4}$ vector for a large-ish $n$, and hence the ratio $t_{n+4} / t_{n+3}$ = `t[1]/t[2]`.   (Doesn't need to be too big because of the exponential growth.  Here, the `big` function is used to tell Julia to switch to slow [arbitrary-precision numbers](https://en.wikipedia.org/wiki/Arbitrary-precision_arithmetic) so that you don't run out of digits when $n$ gets big.)

### Solution
#### (a)
We choose the matrix
$$A\begin{pmatrix} 1 & 1 & 1 & 1\\ 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0\\ 0 & 0 & 0 & 1\end{pmatrix}$$
so that we have
$$A\begin{pmatrix} t_{n+3}\\ t_{n+2}\\ t_{n+1}\\ t_{n}\end{pmatrix}=\begin{pmatrix} t_{n+3}+t_{n+2}+t_{n+1}+t_n\\ t_{n+3} \\ t_{n+2}\\ t_{n}\end{pmatrix} = \begin{pmatrix} t_{n+4}\\ t_{n+3}\\ t_{n+2}\\ t_{n+1}\end{pmatrix}\,.$$
If we now check numerically with Julia, $A^{16}\begin{pmatrix}1\\0\\0\\0\end{pmatrix}$ should compute $\begin{pmatrix}t_{20}\\t_{19}\\t_{18}\\t_{17}\end{pmatrix}=\begin{pmatrix}20569\\10671\\5536\\2872\end{pmatrix}$.

In [1]:
A=[1 1 1 1
   1 0 0 0
   0 1 0 0
   0 0 1 0]
A^16*[1,0,0,0]

4-element Array{Int64,1}:
 20569
 10671
  5536
  2872

#### (b)
Let us find the eigenvalues of $A$:

In [2]:
eigvals(A)

4-element Array{Complex{Float64},1}:
    1.92756+0.0im     
 -0.0763789+0.814704im
 -0.0763789-0.814704im
  -0.774804+0.0im     

We expect the ratio $t_n/t_{n-1}$ to tend to the value of the biggest eigenvalue (in this case 1.92756). In particular, we always approach such problems by writing the initial vector in the basis of the eigenvectors:
$$ \begin{pmatrix} 1\\0\\0\\0\end{pmatrix} = c_1 v_1+c_2 v_2+c_3 v_3+c_4 v_4$$
where $|\lambda_1|<|\lambda_2|<|\lambda_3|<|\lambda_4|$ are the eigenvalues in order of increasing magnitude, $v_1,\ldots,v_4$ are the corresponding eigenvectors, and $c_1,\ldots,c_4$ are some coefficients.  We then have
$$\begin{pmatrix}t_{n+4}\\t_{n+3}\\t_{n+2}\\t_{n+1}\end{pmatrix}=A^n\begin{pmatrix}1\\0\\0\\0\end{pmatrix} = \lambda_1^n c_1 v_1+\lambda_2^n c_2 v_2+\lambda_3^n c_3 v_3+\lambda_4^n c_4 v_4\approx \lambda_4^n c_4 v_4 $$
since the $\lambda_4^n$ term grows exponentially faster than the other three.   If we look at the last component of this vector, we see $t_{n+1}\approx\lambda_4^n \alpha$, where $\alpha$ is the last component of $c_4 v_4$. Hence
$$ \boxed{ t_{n+1}/t_{n} \approx (\lambda_4^n \alpha)/(\lambda_4^{n-1}\alpha) = \lambda_4=1.92756 }$$
for $n\gg0$.

Let us verify this with Julia:

In [3]:
n=50
t=big(A)^n*[1,0,0,0]
t[1]/t[2]

1.927561975482925303801783836461860202766769057070753895783659858855992063133400

## Problem 2

(From Strang, section 6.4, problem 18.)

Let $A$ be some real rectangular (possibly non-square) matrix.  The block matrix
$$S = \begin{pmatrix} 0 & A \\ A^T & 0 \end{pmatrix}$$
is real-symmetric (where "0" denotes a block of zeros).  Consider the (real) eigenvalues λ and eigenvectors $x = (y,z)$ of S, satisfying $Sx = \lambda x$:

$$
Sx = \underbrace{\begin{pmatrix} 0 & A \\ A^T & 0 \end{pmatrix}}_S \underbrace{\begin{pmatrix} y \\ z \end{pmatrix}}_x = \lambda \begin{pmatrix} y \\ z \end{pmatrix} = \lambda x
$$

**(a)** If $A$ is $m\times n$, how big are the vectors $y$ and $z$, and how big are the two blocks of 0's in $S$?

**(b)** Show that $-\lambda$ is also an eigenvalue, with eigenvector $(y,-z)$.

Check this for a random $3\times 4$ matrix `A = randn(3,4)`, with `S = [ zeros(3,3) A; A' zeros(4,4)]`.  Compute `eigvals(S)`: does it match your prediction?

**(c)** Show that $A^TAz = \lambda^2 z$, so that $\lambda^2$ is an eigenvalue of $A^T A$.  Check this via `eigvals(A'*A)`.

**(d)** If $A = I$ ($2 \times 2$), find all four eigenvectors and eigenvalues of $S$.

### Solution
#### (a)
If $A$ is $m\times n$, the dimensions of the two blocks of zeros are $m\times m$ for the top left one (since it has the same number of rows as $A$) and $n\times n$ for the bottom right one (since it has the same number of columns as $A$). Similarly $y$ is an $m$-dimensional vector and $z$ is an $n$-dimensional one.
#### (b)
If $Sx=\lambda x$, this means that $Az=\lambda y$ and $A^Ty=\lambda z$. Hence
$$S\begin{pmatrix} y\\ -z\end{pmatrix} = \begin{pmatrix}0& A\\ A^T & 0\end{pmatrix}\begin{pmatrix} y\\-z\end{pmatrix} = \begin{pmatrix}-Az\\ A^Ty\end{pmatrix}=\begin{pmatrix} -\lambda y\\ \lambda z\end{pmatrix}=-\lambda\begin{pmatrix} y\\-z\end{pmatrix}\,.$$
So $-\lambda$ is an eigenvalue of $S$ with eigenvector $\begin{pmatrix}y\\-z\end{pmatrix}$. Let us verify this with Julia:

In [4]:
A=randn(3,4)
S=[zeros(3,3) A; A' zeros(4,4)]
λ=eigvals(S)

7-element Array{Float64,1}:
 -4.13035    
 -1.91185    
 -1.14518    
  8.53459e-16
  1.14518    
  1.91185    
  4.13035    

As expected the nonzero eigenvalues (the central eigenvalue is 0 up to machine precision) come in pairs
$\lambda$ and $-\lambda$. In fact 0 has to be an eigenvalue, since $A$ has 7 eigenvalues and the nonzero ones come in pairs.

#### (c)
If $\begin{pmatrix}y\\z\end{pmatrix}$ is an eigenvector of $S$ with eigenvalue $\lambda$, we have $Az=\lambda y$ and $A^Ty=\lambda z$. In particular, $z$ has to be nonzero, since if $z=0$, $y=0$ and $\begin{pmatrix}y\\z\end{pmatrix}=0$ would not be an eigenvector. So
$$ A^TAz = A^T (\lambda y)=\lambda^2z$$
Hence, since $z\neq 0$, we have that $\lambda^2$ is an eigenvalue of $A$ with eigenvector $z$. Let us check it with Julia:

In [5]:
λ²=eigvals(A'*A)
λ²,λ.^2

([2.14651e-16,1.31144,3.65518,17.0598],[17.0598,3.65518,1.31144,7.28392e-31,1.31144,3.65518,17.0598])

and we see that the eigenvalues of $A^TA$ are precisely the squares of the eigenvalues of $A$. (Don't be confused by the fact that we get them in a different order; the ordering of the computed eigenvalues is somewhat arbitrary.)
#### (d)
If $A$ is the $2\times 2$ identity matrix, the only eigenvalue of $A^TA=I$ is 1. So the possible eigenvalues of $S$ are $\pm 1$. In fact if $e_1,e_2$ are two eigenvectors of $A^TA$ (for example $\begin{pmatrix}1\\0\end{pmatrix}$ and $\begin{pmatrix}0\\1\end{pmatrix}$), we have that
$$\begin{pmatrix} e_1\\ Ae_1\end{pmatrix},\ \begin{pmatrix} e_1\\ -Ae_1\end{pmatrix},\ \begin{pmatrix}e_2\\ Ae_2\end{pmatrix},\ \begin{pmatrix} e_2\\ -Ae_2\end{pmatrix}\,,$$
are eigenvectors of $S$ for 1, -1, 1 and -1 respectively. 

Let us verify our computation with Julia


In [6]:
S=[zeros(2,2) eye(2); eye(2) zeros(2,2)]
eigvals(S)

4-element Array{Float64,1}:
 -1.0
 -1.0
  1.0
  1.0

## Problem 3

(From Strang, section 6.4, problem 33.)

Suppose $A^T = -A$, a real *antisymmetric* matrix (also called *skew-symmetric*).   Form a random real antisymmetric $5\times 5$ matrix in Julia via `A = randn(5,5); A = A - A'`.

Explain the following facts about $A$, *and* check each fact numerically for your random `A` matrix:

**(a)** $x^T A x = 0$ for every real vector $x$.  (Try `x'*A*x` in Julia with `x = randn(5)`.)

**(b)** The eigenvalues of $A$ (`eigvals(A)`) are purely imaginary.

**(c)** The determinant of $A$ (`det(A)`) is positive or zero (not negative).

**(d)** If you solve $dx/dt = Ax$ for any initial condition $x(0)$, then the length of $x$ is conserved: $\Vert x(t) \Vert = \Vert x(0) \Vert$ for all $t$.    (In Julia, compare `norm(expm(A*t)*x)` to `norm(x)` for various `t`.)

### Solution
#### (a)
Since $x^TAx$ is a $1\times 1$ matrix, it is equal to its own transpose. But
$$ x^TAx=(x^TAx)^T=x^T A^T (x^T)^T = x^T(-A)x=-x^TAx$$
So $x^TAx$ is a number that is equal to its own opposite: it can be only zero.

Let us verify it with Julia:

In [7]:
A=randn(5,5); A=A-A';
x=randn(5)
x'*A*x

1-element Array{Float64,1}:
 3.33067e-16

Yes, the result is zero (up to round-off errors).

**Alternatively:**  for real vectors, $x^T A x = x \cdot (Ax) = (Ax) \cdot x$, since $x \cdot y = y \cdot x$ for dot products of real vectors.  But that means $x^T A x = (Ax) \cdot x = (Ax)^T x = x^T A^T x = -(x^T A x)$, and as above this means it is zero.

#### (b)

You can prove this very similarly to the proof from class that Hermitian matrices have real eigenvalues.  In particular, suppose we have an eigensolution (possibly complex) $Ax = \lambda x$.  If we take the dot product with $x$, i.e. multiplying both sides by $x^H$, we have:

$$
x^H A x = \lambda x^H x = \lambda \Vert x \Vert^2 \\
= (A^H x)^H x = -(Ax)^H x = -(\lambda x)^H x = -\bar{\lambda} x^H x = -\bar{\lambda} \Vert x \Vert^2
$$

for any matrix with $A^H = -A$ (e.g. real $A$ with $A^T = -A$ as we have here).  But since eigenvectors are nonzero, $\Vert x \Vert^2 > 0$ and the above equation implies:

$$
\boxed{\lambda = -\bar{\lambda} \implies \lambda + \bar{\lambda} = 2 \operatorname{Re} \lambda = 0}
$$

which means that $\lambda$ is purely imaginary (its real part is zero).

**Alternative proof**: If $A^H = -A$, then consider the matrix $B = iA$: $B^H = (iA)^H = \bar{i} A^H = (-i)(-A) = B$, so $B$ is Hermitian.  The eigenvalues of $B$ are therefore purely realy, and since $A = -iB$ it follows that the eigenvalues of $A$ are $-i$ times the (real) eigenvalues of $B$, hence the eigenvalues of $A$ are imaginary.

Let's check it in Julia:

In [8]:
eigvals(A)

5-element Array{Complex{Float64},1}:
  3.29597e-17+3.81676im 
  3.29597e-17-3.81676im 
 -1.21431e-17+0.972632im
 -1.21431e-17-0.972632im
 -1.88558e-17+0.0im     

Yup, the real parts are zero (the tiny real parts are just rounding errors).

#### (c)
The determinant of $A$ is the product of all eigenvalues.  There are two cases:

First, if 0 is an eigenvalue of $A$, then $\det A=0$, which is nonnegative.

Second, if 0 is not an eigenvalue, we still know that all nonreal eigenvalues come in complex conjugate pairs (since $A$ is real).  Since all eigenvalues are purely imaginary, all of the eigenvalues come in conjugate pairs. So if
$$ \lambda_1,\bar\lambda_1,\dots,\lambda_k,\bar\lambda_k$$
are all the eigenvalues of $A$, arranged in complex conjugate pairs, the determinant of $A$ is
$$\det A = \lambda_1\bar\lambda_1\cdots\lambda_k\bar\lambda_k = |\lambda_1|^2\cdots|\lambda_k|^2>0$$
which is positive, as promised.

Let us verify it with Julia:

In [9]:
det(A)

2.9325781323107414e-16

It was zero: because A has an odd size ($5\times 5$), there are two pairs of conjugate eigenvalues and the remaining (nonpaired) eigenvalue must be zero.  The same is true of any odd size.

Let's also try an even-size matrix, which should give a positive determinant:

In [10]:
A=randn(6,6); A=A-A';
det(A)

36.78553497706216

#### (d)
If $\frac{dx}{dt}=At$, then the solution is $x(t) = e^{At} x(0)$.  Hence:

$$
\Vert x(t) \Vert^2 = x(t)^T x(t) = x(0)^T (e^{At})^T e^{At} x(0) = x(0)^T e^{A^T t} e^{At} x(0) =
x(0)^T e^{-A t} e^{At} x(0) = x(0)^T x(0) = \Vert x(0) \Vert^2
$$

as desired.  We have used the fact that $(e^A)^T = e^{A^T}$, which is easily derived from the series expansion $e^A = I + A + A^2/2 +\cdots$.

In fact, this proof shows that $e^{At}$ is a *unitary matrix* (orthogonal matrix) for any $A^H = -A$.

**Alternative proof**: Another way to show that something is constant is to show that the derivative is zero, i.e. we can show $d\Vert x \Vert/dt = 0$.  We can take the derivative simply by the product rule:
$$ \frac{d\lVert x\rVert^2}{dt} = \frac{ d(x^Tx)}{dt}=\left(\frac{dx}{dt}\right)^Tx + x^T\frac{dx}{dt}=(Ax)^Tx + x^TAx = (x^TA^Tx)+x^TAx = 0$$
where the last equality comes from part (a) and from the fact that both $A$ and $A^T=-A$ are antisymmetric. So the derivative of $\lVert x\rVert^2$ is 0, that is the norm squared is a constant function. Hence the norm, its square root, is a constant function too.

Let us verify it with Julia for a randomly chosen $t$:

In [11]:
x=randn(6)
t=rand() # random t in [0,1)
norm(x), norm(expm(A*t)*x)

(2.906887629744583,2.906887629744583)